<a href="https://colab.research.google.com/github/minsungk02/3D_objectdetection/blob/minsung/venv_nuscenes_ipynb(offline_evaluation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GPU/시스템 정보 확인
%%bash
echo "========================================================================"
echo "🖥️ 시스템 정보"
echo "========================================================================"
echo ""
echo "📅 현재 시간: $(date '+%Y-%m-%d %H:%M:%S')"
echo ""
echo "🔧 GPU 정보:"
nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader
echo ""
echo "🔧 CUDA 버전:"
nvcc --version 2>/dev/null | grep "release" || echo "nvcc not found"
echo ""
echo "🔧 Python 버전:"
python --version
echo ""
echo "🔧 디스크 공간:"
df -h /content | tail -1 | awk '{print "   사용: " $3 " / " $2 " (" $5 " 사용중)"}'
echo ""
echo "========================================================================"

🖥️ 시스템 정보

📅 현재 시간: 2026-01-14 06:08:08

🔧 GPU 정보:
NVIDIA L4, 23034 MiB, 550.54.15

🔧 CUDA 버전:
Cuda compilation tools, release 12.5, V12.5.82

🔧 Python 버전:
Python 3.12.12

🔧 디스크 공간:
   사용: 39G / 236G (17% 사용중)



In [ ]:
# Miniconda + UV 설치
%%bash
set -e

echo "========================================================================"
echo "📦 Miniconda + UV 설치"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

# Miniconda 설치
if [ ! -d "/content/conda" ]; then
    echo "1️⃣ Miniconda 다운로드 및 설치..."
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
    bash /tmp/miniconda.sh -b -p /content/conda > /dev/null 2>&1
    rm /tmp/miniconda.sh
    echo "   ✅ Miniconda 설치 완료"
else
    echo "1️⃣ Miniconda 이미 설치됨"
fi

# UV 설치 (base 환경에)
if [ ! -f "/content/conda/bin/uv" ]; then
    echo "2️⃣ UV 설치..."
    /content/conda/bin/pip install uv -q
    echo "   ✅ UV 설치 완료"
else
    echo "2️⃣ UV 이미 설치됨"
fi

echo ""
echo "📋 UV 버전:"
/content/conda/bin/uv --version

echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

📦 Miniconda + UV 설치
📅 시작: 06:08:08

1️⃣ Miniconda 다운로드 및 설치...
   ✅ Miniconda 설치 완료
2️⃣ UV 설치...
   ✅ UV 설치 완료

📋 UV 버전:
uv 0.9.25

📅 완료: 06:08:24


In [ ]:
# Anaconda ToS 동의 + Python 3.10 conda 환경 생성
%%bash
set -e

echo "========================================================================"
echo "🐍 Python 3.10 conda 환경 생성"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

source /content/conda/bin/activate

# Anaconda Terms of Service 동의
echo "1️⃣ Anaconda Terms of Service 동의..."
conda config --set solver classic 2>/dev/null || true
echo "y" | conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main 2>/dev/null || true
echo "y" | conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r 2>/dev/null || true
conda config --add channels conda-forge 2>/dev/null || true
conda config --set channel_priority flexible 2>/dev/null || true
echo "   ✅ 설정 완료"
echo ""

if [ ! -d "/content/conda/envs/openmmlab" ]; then
    echo "2️⃣ openmmlab 환경 생성 중..."
    conda create -n openmmlab python=3.10 -c conda-forge -y -q 2>&1 | tail -5
    echo "   ✅ 환경 생성 완료"
else
    echo "2️⃣ openmmlab 환경 이미 존재"
fi

echo ""
echo "📋 Python 버전 확인:"
/content/conda/envs/openmmlab/bin/python --version
echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

🐍 Python 3.10 conda 환경 생성
📅 시작: 06:08:24

1️⃣ Anaconda Terms of Service 동의...
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
   ✅ 설정 완료

2️⃣ openmmlab 환경 생성 중...


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
   ✅ 환경 생성 완료

📋 Python 버전 확인:
Python 3.10.19

📅 완료: 06:09:01


In [ ]:
# NumPy 1.26.4 설치
%%bash
set -e

echo "========================================================================"
echo "📦 NumPy 1.26.4 설치"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

PY=/content/conda/envs/openmmlab/bin/python
UV=/content/conda/bin/uv

echo "1️⃣ NumPy 설치 중..."
$UV pip install numpy==1.26.4 --python $PY -q

echo ""
echo "📋 NumPy 버전 확인:"
$PY -c "import numpy; print(f'   NumPy: {numpy.__version__}')"
echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

📦 NumPy 1.26.4 설치
📅 시작: 06:09:01

1️⃣ NumPy 설치 중...

📋 NumPy 버전 확인:
   NumPy: 1.26.4

📅 완료: 06:09:02


In [ ]:
# PyTorch 2.1.2 + CUDA 11.8 설치
%%bash
set -e

echo "========================================================================"
echo "🔥 PyTorch 2.1.2 + CUDA 11.8 설치"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

PY=/content/conda/envs/openmmlab/bin/python
UV=/content/conda/bin/uv

echo "1️⃣ PyTorch 설치 중 (약 1분 소요)..."
$UV pip install torch==2.1.2 torchvision==0.16.2 \
    --index-url https://download.pytorch.org/whl/cu118 \
    --python $PY -q

echo ""
echo "📋 설치 확인:"
$PY -c "
import torch
print(f'   PyTorch: {torch.__version__}')
print(f'   CUDA 사용 가능: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   CUDA 버전: {torch.version.cuda}')
"

echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

🔥 PyTorch 2.1.2 + CUDA 11.8 설치
📅 시작: 06:09:02

1️⃣ PyTorch 설치 중 (약 1분 소요)...

📋 설치 확인:
   PyTorch: 2.1.2+cu118
   CUDA 사용 가능: True
   GPU: NVIDIA L4
   CUDA 버전: 11.8

📅 완료: 06:09:33


In [ ]:
# OpenMMLab 스택 설치 (mmcv, mmengine, mmdet)
%%bash
set -e

echo "========================================================================"
echo "📦 OpenMMLab 스택 설치"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

PY=/content/conda/envs/openmmlab/bin/python
UV=/content/conda/bin/uv

echo "1️⃣ mmcv 2.1.0 설치 중..."
$UV pip install mmcv==2.1.0 \
    -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html \
    --python $PY -q
echo "   ✅ mmcv 설치 완료"

echo "2️⃣ mmengine 설치 중..."
$UV pip install mmengine==0.10.7 --python $PY -q
echo "   ✅ mmengine 설치 완료"

echo "3️⃣ mmdet 설치 중..."
$UV pip install mmdet==3.3.0 --python $PY -q
echo "   ✅ mmdet 설치 완료"

echo ""
echo "📋 설치 확인:"
$PY -c "
import mmcv
import mmengine
import mmdet
print(f'   mmcv: {mmcv.__version__}')
print(f'   mmengine: {mmengine.__version__}')
print(f'   mmdet: {mmdet.__version__}')
"

echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

📦 OpenMMLab 스택 설치
📅 시작: 06:09:34

1️⃣ mmcv 2.1.0 설치 중...
   ✅ mmcv 설치 완료
2️⃣ mmengine 설치 중...
   ✅ mmengine 설치 완료
3️⃣ mmdet 설치 중...
   ✅ mmdet 설치 완료

📋 설치 확인:
   mmcv: 2.1.0
   mmengine: 0.10.7
   mmdet: 3.3.0

📅 완료: 06:09:42


In [ ]:
%%bash
set -e

echo "========================================================================"
echo "📦 MMDetection3D v1.4.0 설치 (Build Isolation 해제)"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

PY=/content/conda/envs/openmmlab/bin/python
PIP=/content/conda/envs/openmmlab/bin/pip
UV=/content/conda/bin/uv

cd /content

# 1. 클론
if [ ! -d "mmdetection3d" ]; then
    echo "1️⃣ MMDetection3D 클론 중..."
    git clone -b v1.4.0 --depth 1 https://github.com/open-mmlab/mmdetection3d.git 2>/dev/null
    echo "   ✅ 클론 완료"
else
    echo "1️⃣ MMDetection3D 이미 존재"
fi

# 2. 필수 라이브러리(PyTorch 등) 사전 확인 및 설치
echo "2️⃣ 환경 점검 및 의존성 설치..."
# setup.py 실행 전 torch가 반드시 있어야 함. 없으면 설치.
$PIP install torch torchvision --index-url https://download.pytorch.org/whl/cu118 --no-cache-dir || true
$UV pip install -r mmdetection3d/requirements/runtime.txt --python $PY -q 2>/dev/null || true
$UV pip install "nuscenes-devkit==1.1.11" --python $PY -q 2>/dev/null || true
echo "   ✅ 의존성 준비 완료"

# 3. np.long 패치
echo "3️⃣ NumPy 호환성 패치..."
find /content/mmdetection3d -name "*.py" -exec grep -l "np\.long[^a-z]" {} \; 2>/dev/null | while read f; do
    sed -i 's/np\.long\([^a-z]\)/np.int64\1/g' "$f"
done
echo "   ✅ 패치 완료"

# ==========================================================
# ★ 핵심 수정: --no-build-isolation 추가 ★
# 이 옵션이 없으면 pip가 격리된 공간에서 빌드하려다 torch를 못 찾아 에러가 납니다.
# ==========================================================
echo "4️⃣ MMDetection3D 본체 설치 (Editable + No Isolation)..."
cd mmdetection3d
$PIP install -v -e . --no-build-isolation --no-cache-dir

echo ""
echo "✅ 설치 완료!"

📦 MMDetection3D v1.4.0 설치 (Build Isolation 해제)
📅 시작: 06:09:42

1️⃣ MMDetection3D 클론 중...
   ✅ 클론 완료
2️⃣ 환경 점검 및 의존성 설치...
Looking in indexes: https://download.pytorch.org/whl/cu118
   ✅ 의존성 준비 완료
3️⃣ NumPy 호환성 패치...
   ✅ 패치 완료
4️⃣ MMDetection3D 본체 설치 (Editable + No Isolation)...
Using pip 25.3 from /content/conda/envs/openmmlab/lib/python3.10/site-packages/pip (python 3.10)
Obtaining file:///content/mmdetection3d
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for mmdet3d (pyproject.toml): started
  Building editable for mmdet3d (pyproject.toml): finished with status 'done'
  Created wheel for mmdet3d: filename=mmdet3d-1.4.0-0.editable-py3-none-any.whl size=13137 sha256=c8bffd5be63de0020373ec0ce35e1e9c34b44e343d93d4372e220c38028d8efe
  Stored i

  Running command Checking if build backend supports build_editable
  Running command Preparing editable metadata (pyproject.toml)
  /content/conda/envs/openmmlab/lib/python3.10/site-packages/torch/utils/cpp_extension.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
    from pkg_resources import packaging  # type: ignore[attr-defined]
  /content/conda/envs/openmmlab/lib/python3.10/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!

          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:

          License :: OSI Approved :: Apache Software License

          See https://packaging.python.org/en/latest

In [ ]:
# numba 설정 (CUDA 비활성화 + CPU fallback)
%%bash
set -e

echo "========================================================================"
echo "🔧 numba 설정 (CUDA 호환성 문제 해결)"
echo "========================================================================"
echo "📅 시작: $(date '+%H:%M:%S')"
echo ""

PY=/content/conda/envs/openmmlab/bin/python
UV=/content/conda/bin/uv
export PYTHONPATH=/content/mmdetection3d:$PYTHONPATH

# numba 0.57.1 설치 (중간 버전)
echo "1️⃣ numba 0.57.1 + llvmlite 0.40.1 설치..."
$UV pip uninstall numba llvmlite -y --python $PY -q 2>/dev/null || true
$UV pip install numba==0.57.1 llvmlite==0.40.1 --python $PY -q
echo "   ✅ numba 0.57.1 설치 완료"

# numba 캐시 삭제
echo ""
echo "2️⃣ numba 캐시 삭제..."
rm -rf ~/.cache/numba
rm -rf /root/.cache/numba
rm -rf /content/mmdetection3d/mmdet3d/evaluation/functional/kitti_utils/__pycache__
find /content/mmdetection3d -name "*.nbc" -delete 2>/dev/null || true
find /content/mmdetection3d -name "*.nbi" -delete 2>/dev/null || true
echo "   ✅ 캐시 삭제 완료"

# ⚠️ 핵심: numba CUDA 비활성화 설정 파일 생성
echo ""
echo "3️⃣ numba CUDA 비활성화 설정..."
mkdir -p /content/conda/envs/openmmlab/etc/conda/activate.d
cat > /content/conda/envs/openmmlab/etc/conda/activate.d/env_vars.sh << 'ENVEOF'
#!/bin/bash
export NUMBA_DISABLE_CUDA=1
ENVEOF
chmod +x /content/conda/envs/openmmlab/etc/conda/activate.d/env_vars.sh
echo "   ✅ NUMBA_DISABLE_CUDA=1 설정 완료"

echo ""
echo "4️⃣ 의존성 확인:"
$PY << 'EOF'
import os
os.environ['NUMBA_DISABLE_CUDA'] = '1'  # Python 내에서도 설정

import sys
print(f"   Python: {sys.version.split()[0]}")

import numpy as np
print(f"   NumPy: {np.__version__}")

import torch
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")

import mmcv
print(f"   mmcv: {mmcv.__version__}")

import mmengine
print(f"   mmengine: {mmengine.__version__}")

import mmdet
print(f"   mmdet: {mmdet.__version__}")

import mmdet3d
print(f"   mmdet3d: {mmdet3d.__version__}")

import numba
print(f"   numba: {numba.__version__}")
print(f"   NUMBA_DISABLE_CUDA: {os.environ.get('NUMBA_DISABLE_CUDA', 'not set')}")

import llvmlite
print(f"   llvmlite: {llvmlite.__version__}")

if torch.cuda.is_available():
    print(f"\n   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
EOF

echo ""
echo "📅 완료: $(date '+%H:%M:%S')"
echo "========================================================================"

🔧 numba 설정 (CUDA 호환성 문제 해결)
📅 시작: 06:10:00

1️⃣ numba 0.57.1 + llvmlite 0.40.1 설치...
   ✅ numba 0.57.1 설치 완료

2️⃣ numba 캐시 삭제...
   ✅ 캐시 삭제 완료

3️⃣ numba CUDA 비활성화 설정...
   ✅ NUMBA_DISABLE_CUDA=1 설정 완료

4️⃣ 의존성 확인:
   Python: 3.10.19
   NumPy: 1.24.4
   PyTorch: 2.1.2+cu118
   CUDA available: True
   mmcv: 2.1.0
   mmengine: 0.10.7
   mmdet: 3.3.0
   mmdet3d: 1.4.0
   numba: 0.57.1
   NUMBA_DISABLE_CUDA: 1
   llvmlite: 0.40.1

   GPU: NVIDIA L4
   GPU Memory: 22.2 GB

📅 완료: 06:10:04


## [셀 1] Drive 마운트 + 경로 확인 + 데이터셋 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import glob, os

cands = glob.glob("/content/drive/MyDrive/**/v1.0-mini.tgz", recursive=True)
print("Found:", len(cands))
for p in cands[:20]:
    print(" -", p)

assert cands, "Drive에서 v1.0-mini.tgz를 못 찾았습니다. 파일명/위치를 확인해주세요."
TGZ_PATH = cands[0]
print("\nUsing TGZ_PATH =", TGZ_PATH)
print("File size (GB):", os.path.getsize(TGZ_PATH)/1024**3)

Mounted at /content/drive
Found: 1
 - /content/drive/MyDrive/LIKE_LION/1st_project(3D object detection)/v1.0-mini.tgz

Using TGZ_PATH = /content/drive/MyDrive/LIKE_LION/1st_project(3D object detection)/v1.0-mini.tgz
File size (GB): 3.881890502758324


In [ ]:
import os, shutil

LOCAL_TGZ = "/content/v1.0-mini.tgz"
if os.path.exists(LOCAL_TGZ):
    print("Already exists:", LOCAL_TGZ, "-> skip copy")
else:
    print("Copying to:", LOCAL_TGZ)
    shutil.copy2(TGZ_PATH, LOCAL_TGZ)
    print("Copy done.")

print("Local file size (GB):", os.path.getsize(LOCAL_TGZ)/1024**3)

Copying to: /content/v1.0-mini.tgz
Copy done.
Local file size (GB): 3.881890502758324


In [ ]:
import os, subprocess, shlex

NUSC_LOCAL_ROOT = "/content/nuscenes"
os.makedirs(NUSC_LOCAL_ROOT, exist_ok=True)
print("NUSC_LOCAL_ROOT =", NUSC_LOCAL_ROOT)

def sh(cmd):
    print("\n[RUN]", cmd)
    r = subprocess.run(["bash","-lc", cmd], text=True)
    if r.returncode != 0:
        raise RuntimeError(f"Command failed: {r.returncode}")

# 이미 풀려있는지 체크
need = ["maps", "samples", "sweeps", "v1.0-mini"]
already_ok = all(os.path.isdir(os.path.join(NUSC_LOCAL_ROOT, n)) for n in need)
print("Already extracted OK?:", already_ok)

if not already_ok:
    sh(f'tar -xzf "{LOCAL_TGZ}" -C "{NUSC_LOCAL_ROOT}"')
    print("Extracted.")

NUSC_LOCAL_ROOT = /content/nuscenes
Already extracted OK?: False

[RUN] tar -xzf "/content/v1.0-mini.tgz" -C "/content/nuscenes"
Extracted.


In [ ]:
import os

must = ["maps", "samples", "sweeps", "v1.0-mini"]
for m in must:
    p = os.path.join(NUSC_LOCAL_ROOT, m)
    print(f"{m:10s}", "OK" if os.path.isdir(p) else "MISSING", "->", p)

print("\nTop-level listing:", sorted(os.listdir(NUSC_LOCAL_ROOT))[:50])

# samples 내부에 실제 파일이 있는지 간단 체크
samples_dir = os.path.join(NUSC_LOCAL_ROOT, "samples")
if os.path.isdir(samples_dir):
    # CAM_FRONT 예시 이미지가 존재하는지
    cam_front = os.path.join(samples_dir, "CAM_FRONT")
    if os.path.isdir(cam_front):
        ex = sorted(os.listdir(cam_front))[:5]
        print("\nExample CAM_FRONT files:", ex)
    else:
        print("\nNo CAM_FRONT folder under samples (unexpected for full mini package).")

maps       OK -> /content/nuscenes/maps
samples    OK -> /content/nuscenes/samples
sweeps     OK -> /content/nuscenes/sweeps
v1.0-mini  OK -> /content/nuscenes/v1.0-mini

Top-level listing: ['.v1.0-mini.txt', 'LICENSE', 'maps', 'samples', 'sweeps', 'v1.0-mini']

Example CAM_FRONT files: ['n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151603512404.jpg', 'n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604012404.jpg', 'n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151604512404.jpg', 'n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605012404.jpg', 'n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151605512404.jpg']


## mmdetection3d repo의 data/nuscenes를 로컬 데이터로 연결(symlink)


In [ ]:
import os
import subprocess

# 1. 경로 변수 확정
MM3D_REPO = "/content/mmdetection3d"
LOCAL_NUSC_ROOT = "/content/nuscenes"  # 압축 해제된 실제 위치

# 2. 폴더가 실제로 있는지 확인
if not os.path.exists(LOCAL_NUSC_ROOT):
    raise FileNotFoundError(f"❌ 데이터가 {LOCAL_NUSC_ROOT}에 없습니다. 압축 해제 단계를 다시 확인해주세요.")

if not os.path.exists(MM3D_REPO):
    raise FileNotFoundError(f"❌ {MM3D_REPO} 폴더가 없습니다. 설치가 제대로 안 된 것 같습니다.")

# 3. data 폴더 생성
os.makedirs(os.path.join(MM3D_REPO, "data"), exist_ok=True)

# 4. 심볼릭 링크 생성 (기존에 있으면 지우고 다시 연결)
# ln -sfn: f(force), n(no-dereference) - 기존 링크 덮어쓰기
cmd = f"ln -sfn {LOCAL_NUSC_ROOT} {MM3D_REPO}/data/nuscenes"
subprocess.run(cmd, shell=True, check=True)

print(f"✅ Symlink 연결 완료!")
print(f"   원본: {LOCAL_NUSC_ROOT}")
print(f"   타겟: {MM3D_REPO}/data/nuscenes")

# 5. 연결 상태 확인
print("\n[연결된 폴더 내부 확인]")
subprocess.run(f"ls -F {MM3D_REPO}/data/nuscenes | head -n 5", shell=True)

✅ Symlink 연결 완료!
   원본: /content/nuscenes
   타겟: /content/mmdetection3d/data/nuscenes

[연결된 폴더 내부 확인]


CompletedProcess(args='ls -F /content/mmdetection3d/data/nuscenes | head -n 5', returncode=0)

## conda env python으로 전처리(create_data.py) 실행


In [ ]:
%%bash
set -e

# 1. 작업 경로 이동
cd /content/mmdetection3d

# 2. PYTHONPATH 설정
export PYTHONPATH=/content/mmdetection3d:$PYTHONPATH

# 3. 파이썬 경로 지정
PY="/content/conda/envs/openmmlab/bin/python"

echo "========================================================================"
echo "🛠️ NuScenes 데이터 전처리 (create_data.py) - 로그 출력 모드"
echo "========================================================================"
echo "ℹ️ 실행 Python: $PY"
echo "ℹ️ 옵션: -u (실시간 로그 출력)"
echo ""

# 4. create_data.py 실행
# 👉 [변경점] $PY 뒤에 '-u'를 추가하여 로그가 즉시 출력되도록 설정
$PY -u tools/create_data.py nuscenes \
    --root-path ./data/nuscenes \
    --out-dir ./data/nuscenes \
    --extra-tag nuscenes \
    --version v1.0-mini

echo ""
echo "========================================================================"
echo "🎉 전처리 완료! 생성된 .pkl 파일 확인:"
ls -lh ./data/nuscenes/*.pkl
echo "========================================================================"

🛠️ NuScenes 데이터 전처리 (create_data.py) - 로그 출력 모드
ℹ️ 실행 Python: /content/conda/envs/openmmlab/bin/python
ℹ️ 옵션: -u (실시간 로그 출력)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.690 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
total scene num: 10
exist scene num: 10
train scene: 8, val scene: 2
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 404/404, 11.5 task/s, elapsed: 35s, ETA:     0s
train sample: 323, val sample: 81
./data/nuscenes/nuscenes_infos_train.pkl will be modified.
Warning, you may overwriting the original data ./data/nuscenes/nuscenes_infos_train.pkl.
Reading from input file: ./data/nuscenes/nuscenes_infos_train.pkl.
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,


In [ ]:
import glob, os
pkl_list = sorted(glob.glob("/content/mmdetection3d/data/nuscenes/*.pkl"))
print("PKL files:", len(pkl_list))
for p in pkl_list:
    print(" -", os.path.basename(p))
assert pkl_list, "pkl 생성이 안 됐습니다. create_data.py 로그를 확인하세요."

PKL files: 3
 - nuscenes_dbinfos_train.pkl
 - nuscenes_infos_train.pkl
 - nuscenes_infos_val.pkl


## PointPillars config 선택 + mini용 config 생성


In [ ]:
import os

PY = "/content/conda/envs/openmmlab/bin/python"
MM3D = "/content/mmdetection3d"
NUSC_LOCAL_ROOT = "/content/nuscenes"  # 방금 압축 푼 로컬 경로

assert os.path.isfile(PY), "openmmlab python이 없습니다. conda env 생성/설치 셀을 먼저 확인하세요."
assert os.path.isdir(MM3D), "mmdetection3d repo가 없습니다. clone 셀을 먼저 확인하세요."
assert os.path.isdir(NUSC_LOCAL_ROOT), "nuScenes 로컬 데이터가 없습니다. 압축 해제 셀을 먼저 확인하세요."

print("PY =", PY)
print("MM3D =", MM3D)
print("NUSC_LOCAL_ROOT =", NUSC_LOCAL_ROOT)

PY = /content/conda/envs/openmmlab/bin/python
MM3D = /content/mmdetection3d
NUSC_LOCAL_ROOT = /content/nuscenes


In [ ]:
%%writefile /content/mmdetection3d/my_train_long_config.py

# 1. Base Config 상속
_base_ = ['configs/pointpillars/pointpillars_hv_fpn_sbn-all_8xb4-2x_nus-3d.py']

# 2. 기본 경로
data_root = 'data/nuscenes/'
work_dir = '/content/drive/MyDrive/nuscenes_outputs/work_dirs/pointpillars_train_long4'

# 3. 학습 데이터셋 설정 (Mini)
train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    dataset=dict(
        data_root=data_root,
        ann_file='nuscenes_infos_train.pkl',
        pipeline=[
            dict(type='LoadPointsFromFile', coord_type='LIDAR', load_dim=5, use_dim=5, backend_args=None),
            dict(type='LoadPointsFromMultiSweeps', sweeps_num=10, backend_args=None),
            dict(type='LoadAnnotations3D', with_bbox_3d=True, with_label_3d=True),
            dict(type='GlobalRotScaleTrans', rot_range=[-0.3925, 0.3925], scale_ratio_range=[0.95, 1.05], translation_std=[0, 0, 0]),
            dict(type='RandomFlip3D', flip_ratio_bev_horizontal=0.5),
            dict(type='PointsRangeFilter', point_cloud_range=[-50, -50, -5, 50, 50, 3]),
            dict(type='ObjectRangeFilter', point_cloud_range=[-50, -50, -5, 50, 50, 3]),
            dict(type='ObjectNameFilter', classes=['car', 'truck', 'trailer', 'bus', 'construction_vehicle', 'bicycle', 'motorcycle', 'pedestrian', 'traffic_cone', 'barrier']),
            dict(type='PointShuffle'),
            dict(type='Pack3DDetInputs', keys=['points', 'gt_bboxes_3d', 'gt_labels_3d'])
        ]
    )
)

# 4. 검증 끄기 (학습 중 에러 방지)
val_cfg = None
val_dataloader = None
val_evaluator = None

# 5. 테스트 데이터셋 (평가용)
test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        data_root=data_root,
        ann_file='nuscenes_infos_val.pkl',
        pipeline=[
            dict(type='LoadPointsFromFile', coord_type='LIDAR', load_dim=5, use_dim=5, backend_args=None),
            dict(type='LoadPointsFromMultiSweeps', sweeps_num=10, test_mode=True, backend_args=None),
            dict(type='MultiScaleFlipAug3D', img_scale=(1333, 800), pts_scale_ratio=1, flip=False,
                 transforms=[
                     dict(type='GlobalRotScaleTrans', rot_range=[0, 0], scale_ratio_range=[1.0, 1.0], translation_std=[0, 0, 0]),
                     dict(type='RandomFlip3D'),
                     dict(type='PointsRangeFilter', point_cloud_range=[-50, -50, -5, 50, 50, 3])
                 ]),
            dict(type='Pack3DDetInputs', keys=['points'])
        ]
    )
)

# 6. 평가 설정 (Format Only 모드용)
test_evaluator = dict(
    type='NuScenesMetric',
    data_root=data_root,
    ann_file=data_root + 'nuscenes_infos_val.pkl',
    metric='bbox',
    backend_args=None
)

# 7. 학습 설정 (30 Epochs, 5 Epoch 마다 저장)
train_cfg = dict(max_epochs=30, val_interval=9999)
default_hooks = dict(
    checkpoint=dict(type='CheckpointHook', interval=5, by_epoch=True)
)

Overwriting /content/mmdetection3d/my_train_long_config.py


In [ ]:
%%bash
set -e
cd /content/mmdetection3d
export PYTHONPATH=/content/mmdetection3d:$PYTHONPATH
PY="/content/conda/envs/openmmlab/bin/python"

echo "========================================================================"
echo "🚀 30 Epoch 학습 시작 (5 Epoch 마다 저장)"
echo "========================================================================"

$PY -u tools/train.py my_train_long_config.py

🚀 30 Epoch 학습 시작 (5 Epoch 마다 저장)
01/14 06:17:00 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:29:10) [GCC 14.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1128668475
    GPU 0: NVIDIA L4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0
    PyTorch: 2.1.2+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.8
  - NVCC arch

/content/conda/envs/openmmlab/lib/python3.10/site-packages/torch/utils/cpp_extension.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging  # type: ignore[attr-defined]
/content/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/content/conda/envs/openmmlab/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/conda/envs/openmmlab/lib/python3.10/site-packages/mmdet/mo

# 📊 3단계: [자동화] Best Model 선정 (mAP 평가)

In [ ]:
%%bash
set -e
cd /content/mmdetection3d
export PYTHONPATH=/content/mmdetection3d:$PYTHONPATH
PY="/content/conda/envs/openmmlab/bin/python"

WORK_DIR="/content/drive/MyDrive/nuscenes_outputs/work_dirs/pointpillars_train_long4"
TEMP_JSON_DIR="$WORK_DIR/temp_results"
mkdir -p $TEMP_JSON_DIR

echo "========================================================================"
echo "🏆 Best Model 선정 작업 시작 (경로 자동 탐색 적용)"
echo "========================================================================"

# 저장된 epoch_*.pth 파일들 찾기 (숫자 정렬)
CHECKPOINTS=$(ls $WORK_DIR/epoch_*.pth | sort -V)

BEST_SCORE=0
BEST_EPOCH=""

for CKPT in $CHECKPOINTS; do
    EPOCH_NAME=$(basename $CKPT)
    echo "--------------------------------------------------------"
    echo "🔍 평가 중: $EPOCH_NAME"

    # 1. JSON 생성 (Format Only)
    # 각 체크포인트마다 별도의 폴더에 JSON 생성
    CURRENT_TEMP_DIR="$TEMP_JSON_DIR/${EPOCH_NAME%.*}"
    mkdir -p $CURRENT_TEMP_DIR

    $PY tools/test.py my_train_long_config.py $CKPT \
        --cfg-options test_evaluator.format_only=True \
        test_evaluator.jsonfile_prefix=$CURRENT_TEMP_DIR/results_nusc > /dev/null 2>&1

    # ★ 수정된 부분: 정확한 경로를 예측하는 대신, find로 찾아버림 ★
    JSON_FILE=$(find $CURRENT_TEMP_DIR -name "*.json" | head -n 1)

    if [ -z "$JSON_FILE" ]; then
        echo "   ⚠️ JSON 생성 실패 (Skip)"
        continue
    fi

    # 2. NuScenes Devkit으로 채점
    SCORE=$($PY -c "
from nuscenes import NuScenes
from nuscenes.eval.detection.config import config_factory
from nuscenes.eval.detection.evaluate import NuScenesEval
import json
import sys

# 표준 출력(stdout)을 막아서 불필요한 로그 숨김
import os
sys.stdout = open(os.devnull, 'w')

try:
    nusc = NuScenes(version='v1.0-mini', dataroot='data/nuscenes', verbose=False)
    cfg = config_factory('detection_cvpr_2019')
    # verbose=False로 설정
    nusc_eval = NuScenesEval(nusc, config=cfg, result_path='$JSON_FILE', eval_set='mini_val', output_dir='$CURRENT_TEMP_DIR', verbose=False)
    metrics, _ = nusc_eval.evaluate()

    # 다시 표준 출력을 원상복구해서 점수만 출력
    sys.stdout = sys.__stdout__
    print(metrics.nd_score)
except Exception as e:
    sys.stdout = sys.__stdout__
    print(0.0)
")

    echo "   👉 NDS Score: $SCORE"

    # 3. 최고 점수 갱신 확인
    if (( $(echo "$SCORE > $BEST_SCORE" | bc -l) )); then
        BEST_SCORE=$SCORE
        BEST_EPOCH=$CKPT
        echo "   🌟 현재 최고 점수 갱신!"
    fi
done

echo "========================================================================"
if [ -z "$BEST_EPOCH" ]; then
    echo "❌ 실패: 점수가 모두 0.0이거나 JSON 생성에 실패했습니다."
else
    echo "👑 [최종 결과] Best Model: $(basename $BEST_EPOCH)"
    echo "👑 Best NDS Score: $BEST_SCORE"

    # Best Model 경로 저장
    echo $BEST_EPOCH > best_model_path.txt
fi
echo "========================================================================"

🏆 Best Model 선정 작업 시작 (경로 자동 탐색 적용)
--------------------------------------------------------
🔍 평가 중: epoch_5.pth
   👉 NDS Score: 0.02657416075356949
   🌟 현재 최고 점수 갱신!
--------------------------------------------------------
🔍 평가 중: epoch_10.pth
   👉 NDS Score: 0.03297834316784153
   🌟 현재 최고 점수 갱신!
--------------------------------------------------------
🔍 평가 중: epoch_15.pth
   👉 NDS Score: 0.08584949143927093
   🌟 현재 최고 점수 갱신!
--------------------------------------------------------
🔍 평가 중: epoch_20.pth
   👉 NDS Score: 0.09892546944268023
   🌟 현재 최고 점수 갱신!
--------------------------------------------------------
🔍 평가 중: epoch_25.pth
   👉 NDS Score: 0.10658523756445762
   🌟 현재 최고 점수 갱신!
--------------------------------------------------------
🔍 평가 중: epoch_30.pth
   👉 NDS Score: 0.1054342927231066
👑 [최종 결과] Best Model: epoch_25.pth
👑 Best NDS Score: 0.10658523756445762


# 🎁 4단계: 최종 결과 추출 (Best Model로 JSON 생성)

In [ ]:
%%bash
set -e
cd /content/mmdetection3d
export PYTHONPATH=/content/mmdetection3d:$PYTHONPATH
PY="/content/conda/envs/openmmlab/bin/python"

# Best Model 경로 읽기
BEST_CHECKPOINT=$(cat best_model_path.txt)
echo "✅ 최종 선택된 모델: $BEST_CHECKPOINT"

# 결과 저장 경로
FINAL_RESULT_DIR="/content/drive/MyDrive/nuscenes_outputs/final_best_results2"
mkdir -p $FINAL_RESULT_DIR

echo "========================================================================"
echo "🎁 최종 Best Model의 결과 JSON 생성"
echo "========================================================================"

$PY tools/test.py my_train_long_config.py $BEST_CHECKPOINT \
    --cfg-options \
    test_evaluator.format_only=True \
    test_evaluator.jsonfile_prefix=$FINAL_RESULT_DIR/results_nusc

echo ""
echo "🎉 모든 작업 완료! 최종 결과 파일:"
ls -lh $FINAL_RESULT_DIR/results_nusc/pred_instances_3d/*.json

✅ 최종 선택된 모델: /content/drive/MyDrive/nuscenes_outputs/work_dirs/pointpillars_train_long4/epoch_25.pth
🎁 최종 Best Model의 결과 JSON 생성
01/14 07:18:15 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:29:10) [GCC 14.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 905008515
    GPU 0: NVIDIA L4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0
    PyTorch: 2.1.2+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by 

/content/conda/envs/openmmlab/lib/python3.10/site-packages/torch/utils/cpp_extension.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging  # type: ignore[attr-defined]
/content/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/content/mmdetection3d/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-ba

In [ ]:
## 나머지 정리 코드 불필요한 거 제거 ##

%%bash
set -e
# ... (이전 단계: Best Model 선정 및 JSON 생성 완료 후) ...

# 경로는 아까 설정한 변수들을 그대로 씁니다.
# WORK_DIR: 지저분한 작업 폴더
# FINAL_RESULT_DIR: 최종 결과 폴더
# BEST_CHECKPOINT: 1등 모델 경로

echo "========================================================================"
echo "🧹 [Clean Up] 파이프라인 정리 작업 시작"
echo "========================================================================"

# 1. 진짜 필요한 '최종 산출물 폴더' 만들기
DELIVERABLES_DIR="/content/drive/MyDrive/nuscenes_outputs/FINAL_DELIVERABLES"
mkdir -p $DELIVERABLES_DIR

echo "📦 1. 1등 모델($BEST_CHECKPOINT)을 안전한 곳으로 이동 중..."
cp $BEST_CHECKPOINT $DELIVERABLES_DIR/best_model.pth

echo "📦 2. 최종 결과 JSON을 안전한 곳으로 이동 중..."
# 위에서 생성한 최종 json 폴더를 통째로 복사
cp -r $FINAL_RESULT_DIR/* $DELIVERABLES_DIR/

echo "📦 3. Config 파일 백업"
cp /content/mmdetection3d/my_train_long_config.py $DELIVERABLES_DIR/train_config.py

echo "------------------------------------------------------------------------"
echo "🗑️ 4. 불필요한 파일 삭제 (탈락한 모델들 & 임시 파일)"

# (선택 1) 과감하게 작업 폴더를 통째로 날리고 싶다면 아래 주석 해제
# rm -rf $WORK_DIR
# echo "   👉 작업 폴더($WORK_DIR) 전체 삭제 완료!"

# (선택 2) 1등 모델 빼고 나머지 pth만 지우기 (로그는 남김) - 추천!
find $WORK_DIR -name "epoch_*.pth" ! -name "$(basename $BEST_CHECKPOINT)" -delete
rm -rf $WORK_DIR/temp_results  # 임시 JSON 폴더 삭제

echo "   👉 탈락한 모델(.pth) 및 임시 JSON 파일 삭제 완료!"
echo "========================================================================"
echo "✨ 최종 정리 완료! 여기만 보시면 됩니다: $DELIVERABLES_DIR"
ls -lh $DELIVERABLES_DIR

## 이거 말고 걍 test만 돌려보기 위해 위에 돌릴거임

## 학습 결과



### 1️⃣ 현재 상황 정리: 우리는 무엇을 얻었나?

지금 님 손에는 두 가지 보물이 쥐어져 있습니다.

1. **`epoch_5.pth` (모델 가중치 파일) = "공부를 마친 학생의 뇌"** 🧠
* 수많은 라이다 데이터를 보고 "아, 점들이 이렇게 생기면 자동차구나"라고 학습한 인공지능 그 자체입니다.
* **용도:** 앞으로 **새로운 데이터**가 들어오면, 이 파일을 불러와서 추론(예측)을 합니다.


2. **`results_nusc.json` (결과 파일) = "학생이 제출한 답안지"** 📝
* 방금 학습한 모델(학생)에게 NuScenes 검증용 문제(Validation Set) 81문제를 풀게 시켰더니, 학생이 써낸 **답안지**입니다.
* **용도:** 이 답안지를 채점(Evaluation)해서 점수를 매기거나, 답안을 눈으로 확인(Visualization)할 때 씁니다.



---

### 2️⃣ JSON 파일 뜯어보기: 대체 뭐가 들어있나?

이 JSON 파일은 **텍스트**로 된 데이터일 뿐입니다. 열어보면 아래와 같은 정보가 수천 개 적혀 있습니다.

> **"이 장면(Sample)에서, (x, y, z) 위치에, (w, l, h) 크기의, '자동차'가 있고, 저는 95% 확신합니다."**

실제 구조를 쉽게 풀면 이렇습니다:

```json
{
  "results": {
    "장면_ID_1": [
      {
        "sample_token": "장면_ID_1",
        "translation": [10.5, 20.1, -1.0],  // 위치 (x, y, z) - 내 차 기준 몇 미터 앞/옆인지
        "size": [1.9, 4.8, 1.5],           // 크기 (폭, 길이, 높이) - 차 크기
        "rotation": [0.7, 0.0, 0.0, 0.7],  // 회전 - 차가 어느 방향을 보고 있는지
        "velocity": [0.0, 0.0],            // 속도 (여기선 안 씀)
        "detection_name": "car",           // 종류 - 자동차, 버스, 트럭 등
        "detection_score": 0.95            // 확신도 - 0.0 ~ 1.0 (높을수록 진짜라고 생각함)
      },
      { ...또 다른 자동차 정보... }
    ],
    "장면_ID_2": [ ... ]
  }
}

```

**즉, 이 JSON 파일은 모델이 찾아낸 모든 물체들의 3D 좌표 리스트입니다.**

---

### 3️⃣ 이걸로 무엇을 할 수 있나?

이 JSON 파일(`results_nusc.json`)은 크게 **두 가지 용도**로 쓰입니다.

#### ① 채점 (Evaluation): "너 몇 점이니?"

* 우리는 **정답지(Annotation, `.pkl`)**를 가지고 있습니다.
* **Devkit(채점기)**가 님의 **JSON(제출 답안)**과 **정답지**를 비교합니다.
* 정답: (10, 20) 좌표에 차가 있음.
* 내 모델: (10.5, 20.1) 좌표에 차가 있다고 함.
* 판단: "오, 거의 맞췄네! 정답 인정." -> **mAP 점수 상승** 📈


* 이 과정을 통해 "내 모델의 성능은 **mAP 0.45**입니다"라고 말할 수 있게 됩니다.

#### ② 시각화 (Visualization): "눈으로 확인해보자"

* 사람은 좌표 숫자만 보고는 차를 잘 찾았는지 알 수 없습니다.
* 그래서 아까 우리가 작성한 코드가 이 JSON을 읽어서, **원본 라이다 점(배경) 위에 박스 그림을 그려주는 것**입니다.
* "아, 여기 점들이 모여있는 곳에 정확히 박스를 쳤네!" 하고 눈으로 검증하는 용도입니다.

---

### 4️⃣ (중요) 새로운 데이터셋에 예측을 하려면?

질문하신 **"이걸 바탕으로 새로운 데이터셋에 예측을 하는 건지?"**에 대한 답은 **"아니오, 방식이 다릅니다."** 입니다.

* 지금 만든 `json` 파일은 **"과거의 기록"**일 뿐입니다. (NuScenes Mini 데이터에 대한 결과)
* 만약 님이 **K-City 데이터**나 **직접 찍은 라이다 데이터**를 가져와서 예측하고 싶다면, `json` 파일은 필요 없고 **`epoch_5.pth` (모델 파일)**이 필요합니다.

**[새로운 데이터 예측 프로세스]**

1. **New Data:** 새로운 라이다 데이터(`.bin` 또는 `.pcd`) 준비.
2. **Model Load:** `epoch_5.pth` (학습된 두뇌) 로드.
3. **Inference:** 모델에 새로운 데이터를 넣음.
4. **Output:** 모델이 **새로운 JSON**이나 실시간 좌표를 뱉어냄.

---

### 📝 3줄 요약

1. **학습(Train)**의 결과물은 **모델 파일(`.pth`)**입니다. (이게 진짜 자산입니다.)
2. **검증(Test)**의 결과물은 **JSON 파일**입니다. 이건 "모델이 시험 본 결과지"입니다.
3. **시각화**는 이 JSON(결과지)을 읽어서 그림으로 그려보는 과정입니다.

이제 개념이 잡히셨나요?
그렇다면 이제 **시각화 코드(오류 수정된 버전)**를 돌려서, "내 모델이 시험을 얼마나 잘 쳤나" 눈으로 확인하러 가시죠! 🚀